**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Input

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

> The act function gives the action to do for a given state based on the learned policy. The parameter `epsilon` determines, during training, when to deviate from the learned policy and when to explore new possible actions. It is thanks to this parameter that the algorithm is capable of learning. `epsilon` rules the trade-of between exploration and exploitation.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

# PARAMS

In [5]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=301 # set small when debugging
epochs_test=31 # set small when debugging

In [6]:
# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

> The `position` array determines the current position of the mouse in the game.
>
> The `board` array determines the state of the game, where are the bonus and malus left.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(0, self.n_action, size=1)[0]

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
    loss = 0
        
    for e in range(epochs):
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score += win - lose

        print("Win/lose count {}/{}. Average score ({})"
          .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/18.0. Average score (-9.0)
Win/lose count 9.0/13.0. Average score (-6.5)
Win/lose count 10.5/9.0. Average score (-3.8333333333333335)
Win/lose count 7.5/10.0. Average score (-3.5)
Win/lose count 9.5/14.0. Average score (-3.7)
Win/lose count 6.5/7.0. Average score (-3.1666666666666665)
Win/lose count 8.0/15.0. Average score (-3.7142857142857144)
Win/lose count 9.0/12.0. Average score (-3.625)
Win/lose count 16.0/8.0. Average score (-2.3333333333333335)
Win/lose count 12.0/12.0. Average score (-2.1)
Win/lose count 10.5/17.0. Average score (-2.5)
Win/lose count 8.5/12.0. Average score (-2.5833333333333335)
Win/lose count 9.0/16.0. Average score (-2.923076923076923)
Win/lose count 6.5/8.0. Average score (-2.8214285714285716)
Win/lose count 9.5/11.0. Average score (-2.7333333333333334)
Win/lose count 14.5/18.0. Average score (-2.78125)
Win/lose count 5.0/6.0. Average score (-2.676470588235294)
Win/lose count 14.5/22.0. Average score (-2.9444444444444446)
Win/lose count 6.

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




> Let's show the first equality :
> \begin{align*}
 \mathcal{Q}^\pi(s, a)
  &= \mathbf{E}_{p^\pi} \left[ \sum_{t \le T} \gamma^t r(s_t, a_t) | s_0 = s, a_0 =a \right] \\
  &= \mathbf{E}_{p^\pi} \left[ r(s, a) + \gamma \sum_{1 \le t \le T} \gamma^{t - 1} r(s_t, a_t) | s_0 = s, a_0 = a, (\forall 1 \le t \le T, ~ a_t = \pi(s_t)) \right] \\
  &=  r(s, a) + \gamma \sum_{s'} \mathbf{P}(s_1 = s' | s_0 = s, a_0 = a) \mathbf{E}_{p^\pi} \left[\sum_{1 \le t \le T} \gamma^{t - 1} r(s_t, a_t) | s_1 = s', a_1 = a', (\forall 2 \le t \le T, ~ a_t = \pi(s_t)) \right] \\
  &= r(s, a) + \gamma \mathbf{E}_{(s', a') \sim p(\cdot | s, a)} \left[ \mathcal{Q}^\pi(s', a') \right] \\
  &= \mathbf{E}_{(s', a') \sim p(\cdot | s, a)} \left[ r(s, a) + \gamma \mathcal{Q}^\pi(s', a') \right]
\end{align*}
>
>
> Then, let's show the second equality :
> \begin{align*}
\mathcal{Q}^*(s, a)
    &= \max_\pi \mathcal{Q}^\pi (s, a) \\
    &= \max_\pi \left( r(s, a) + \gamma \mathbf{E}_{(s', a') \sim p(\cdot | s, a, a' = \pi(s'))} \left[ \mathcal{Q}^\pi(s', a') \right] \right) \\
    &= r(s, a) + \gamma \mathbf{E}_{(s') \sim \pi^*(\cdot | s, a)} \left[
    \max_\pi \left( \mathcal{Q}^\pi(s', a') \right) \right] \\
    &= r(s, a) + \gamma \mathbf{E}_{(s') \sim \pi^*(\cdot | s, a)} \left[
    \max_{a'} \left( \mathcal{Q}^*(s', a') \right) \right] \\
    &= \mathbf{E}_{(s') \sim \pi^*(\cdot | s, a)} \left[ r(s, a) + \gamma
    \max_{a'}  \mathcal{Q}^*(s', a') \right] \\
\end{align*}
>
> 
> Finally, we want to have $\mathcal{Q}^*(s, a, \theta)$ close to $\mathbf{E}_{(s') \sim \pi^*(\cdot | s, a)} \left[ r(s, a) + \gamma \max_{a'}  \mathcal{Q}^*(s', a', \theta) \right]$ so we will establish the difference :
> \begin{align*}
\mathcal{L}_{pot}(\theta)
    &= \Vert \mathbf{E}_{(s') \sim \pi^*(\cdot | s, a)} \left[ r(s, a) + \gamma
    \max_{a'}  \mathcal{Q}^*(s', a') \right] - \mathcal{Q}^*(s, a, \theta) \Vert^2 \\
    &= \Vert \sum_{s'} \pi^*(s' | s, a) \left[  \right] r(s, a) + \gamma
    \max_{a'}  \mathcal{Q}^*(s', a')  - \mathcal{Q}^*(s, a, \theta) \Vert^2 \\
    &\le \sum_{s'} \Vert  \pi^*(s' | s, a) \left[ r(s, a) + \gamma
    \max_{a'}  \mathcal{Q}^*(s', a')  - \mathcal{Q}^*(s, a, \theta) \right] \Vert^2 \\
    &\le \sum_{s'} \pi^*(s' | s, a) \Vert  r(s, a) + \gamma
    \max_{a'}  \mathcal{Q}^*(s', a')  - \mathcal{Q}^*(s, a, \theta) \Vert^2 \\
    &\le \mathbf{E}_{(s') \sim \pi^*(\cdot | s, a)} \Vert r(s, a) + \gamma
    \max_{a'}  \mathcal{Q}^*(s', a')  - \mathcal{Q}^*(s, a, \theta) \Vert ^2
\end{align*}
> So $\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r(s, a) +\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$ is a potential objective to minimise.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else:
            self.memory[:-1] = self.memory[1:]
            self.memory[-1] = m

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [11]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        q = self.model.predict(s.reshape(-1, 5, 5, self.n_state)).reshape(-1, 1)
        return np.argmax(q)

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        new_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        
        batch = [self.memory.random_access() for i in range(self.batch_size)]
               
        for i, (state, new_state, _, _, _) in enumerate(batch):
            input_states[i] = state
            new_states[i] = new_state
            
        new_q = self.model.predict(new_states)
        target_q = self.model.predict(input_states)
            
        for i, (_, _, action, reward, game_over) in enumerate(batch):
            if game_over:
                target_q[i, action] = reward
            else:
                target_q[i, action] = reward + self.discount * np.max(new_q[i])
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        
        ####### FILL IN
        width_model = 32
        
        model = Sequential([
            Flatten(input_shape=(5, 5, self.n_state)),
            Dense(width_model, activation='relu'),
            Dense(self.n_action),
        ])
                
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
from keras.layers import Flatten, Input


In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')

Epoch 000/301 | Loss 0.0225 | Win/lose count 4.0/7.0 (-3.0)
Epoch 001/301 | Loss 0.0065 | Win/lose count 2.0/2.0 (0.0)
Epoch 002/301 | Loss 0.0092 | Win/lose count 6.0/5.0 (1.0)
Epoch 003/301 | Loss 0.0040 | Win/lose count 4.0/6.0 (-2.0)
Epoch 004/301 | Loss 0.0031 | Win/lose count 6.0/4.0 (2.0)
Epoch 005/301 | Loss 0.0103 | Win/lose count 3.0/4.0 (-1.0)
Epoch 006/301 | Loss 0.0819 | Win/lose count 1.5/5.0 (-3.5)
Epoch 007/301 | Loss 0.0058 | Win/lose count 4.0/5.0 (-1.0)
Epoch 008/301 | Loss 0.0110 | Win/lose count 5.0/2.0 (3.0)
Epoch 009/301 | Loss 0.0169 | Win/lose count 5.0/3.0 (2.0)
Epoch 010/301 | Loss 0.0136 | Win/lose count 5.0/3.0 (2.0)
Epoch 011/301 | Loss 0.0102 | Win/lose count 9.5/9.0 (0.5)
Epoch 012/301 | Loss 0.0072 | Win/lose count 4.0/2.0 (2.0)
Epoch 013/301 | Loss 0.0117 | Win/lose count 7.5/6.0 (1.5)
Epoch 014/301 | Loss 0.0072 | Win/lose count 6.0/3.0 (3.0)
Epoch 015/301 | Loss 0.0021 | Win/lose count 5.5/5.0 (0.5)
Epoch 016/301 | Loss 0.0061 | Win/lose count 5.5/2.

Epoch 138/301 | Loss 0.0024 | Win/lose count 14.5/0 (14.5)
Epoch 139/301 | Loss 0.0030 | Win/lose count 18.0/2.0 (16.0)
Epoch 140/301 | Loss 0.0015 | Win/lose count 16.5/2.0 (14.5)
Epoch 141/301 | Loss 0.0505 | Win/lose count 18.0/1.0 (17.0)
Epoch 142/301 | Loss 0.0029 | Win/lose count 18.5/3.0 (15.5)
Epoch 143/301 | Loss 0.0090 | Win/lose count 17.0/2.0 (15.0)
Epoch 144/301 | Loss 0.0046 | Win/lose count 20.0/2.0 (18.0)
Epoch 145/301 | Loss 0.0029 | Win/lose count 9.0/3.0 (6.0)
Epoch 146/301 | Loss 0.0031 | Win/lose count 3.5/4.0 (-0.5)
Epoch 147/301 | Loss 0.0538 | Win/lose count 10.5/0 (10.5)
Epoch 148/301 | Loss 0.0536 | Win/lose count 23.0/4.0 (19.0)
Epoch 149/301 | Loss 0.0031 | Win/lose count 12.0/0 (12.0)
Epoch 150/301 | Loss 0.0026 | Win/lose count 13.5/2.0 (11.5)
Epoch 151/301 | Loss 0.0019 | Win/lose count 11.0/2.0 (9.0)
Epoch 152/301 | Loss 0.0031 | Win/lose count 5.5/0 (5.5)
Epoch 153/301 | Loss 0.0080 | Win/lose count 15.5/2.0 (13.5)
Epoch 154/301 | Loss 0.0030 | Win/lose

Epoch 275/301 | Loss 0.0011 | Win/lose count 10.0/4.0 (6.0)
Epoch 276/301 | Loss 0.0531 | Win/lose count 19.0/2.0 (17.0)
Epoch 277/301 | Loss 0.0039 | Win/lose count 1.5/0 (1.5)
Epoch 278/301 | Loss 0.0030 | Win/lose count 13.5/2.0 (11.5)
Epoch 279/301 | Loss 0.0016 | Win/lose count 13.5/1.0 (12.5)
Epoch 280/301 | Loss 0.0029 | Win/lose count 9.0/0 (9.0)
Epoch 281/301 | Loss 0.0014 | Win/lose count 15.5/1.0 (14.5)
Epoch 282/301 | Loss 0.0051 | Win/lose count 18.0/2.0 (16.0)
Epoch 283/301 | Loss 0.0048 | Win/lose count 9.0/1.0 (8.0)
Epoch 284/301 | Loss 0.0028 | Win/lose count 10.0/0 (10.0)
Epoch 285/301 | Loss 0.0035 | Win/lose count 8.0/0 (8.0)
Epoch 286/301 | Loss 0.0037 | Win/lose count 15.5/3.0 (12.5)
Epoch 287/301 | Loss 0.0018 | Win/lose count 19.0/4.0 (15.0)
Epoch 288/301 | Loss 0.0027 | Win/lose count 0.5/2.0 (-1.5)
Epoch 289/301 | Loss 0.0054 | Win/lose count 18.0/5.0 (13.0)
Epoch 290/301 | Loss 0.0154 | Win/lose count 20.0/3.0 (17.0)
Epoch 291/301 | Loss 0.0034 | Win/lose cou

In [14]:
HTML(display_videos('fc_train300.mp4'))

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        
        model = Sequential([
            Conv2D(32, (3, 3), input_shape=(5, 5, self.n_state)),
            Activation('relu'),
            Conv2D(16, (3, 3)),
            Activation('relu'),
            Flatten(),
            Dense(self.n_action),
        ])
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')

Epoch 000/301 | Loss 0.0110 | Win/lose count 7.5/11.0 (-3.5)
Epoch 001/301 | Loss 0.0014 | Win/lose count 2.0/0 (2.0)
Epoch 002/301 | Loss 0.0155 | Win/lose count 6.0/5.0 (1.0)
Epoch 003/301 | Loss 0.0063 | Win/lose count 2.5/0 (2.5)
Epoch 004/301 | Loss 0.0048 | Win/lose count 2.5/2.0 (0.5)
Epoch 005/301 | Loss 0.0030 | Win/lose count 4.0/4.0 (0.0)
Epoch 006/301 | Loss 0.0061 | Win/lose count 3.0/5.0 (-2.0)
Epoch 007/301 | Loss 0.0034 | Win/lose count 6.0/5.0 (1.0)
Epoch 008/301 | Loss 0.0107 | Win/lose count 8.0/3.0 (5.0)
Epoch 009/301 | Loss 0.0060 | Win/lose count 5.0/4.0 (1.0)
Epoch 010/301 | Loss 0.0022 | Win/lose count 3.0/3.0 (0.0)
Epoch 011/301 | Loss 0.0021 | Win/lose count 8.0/3.0 (5.0)
Epoch 012/301 | Loss 0.0015 | Win/lose count 3.5/0 (3.5)
Epoch 013/301 | Loss 0.1058 | Win/lose count 5.5/5.0 (0.5)
Epoch 014/301 | Loss 0.0020 | Win/lose count 5.5/7.0 (-1.5)
Epoch 015/301 | Loss 0.0052 | Win/lose count 5.5/2.0 (3.5)
Epoch 016/301 | Loss 0.0073 | Win/lose count 5.0/4.0 (1.0)

Epoch 137/301 | Loss 0.0099 | Win/lose count 14.5/3.0 (11.5)
Epoch 138/301 | Loss 0.0022 | Win/lose count 8.0/2.0 (6.0)
Epoch 139/301 | Loss 0.0028 | Win/lose count 8.5/2.0 (6.5)
Epoch 140/301 | Loss 0.0033 | Win/lose count 20.0/3.0 (17.0)
Epoch 141/301 | Loss 0.0518 | Win/lose count 14.5/4.0 (10.5)
Epoch 142/301 | Loss 0.0019 | Win/lose count 9.0/1.0 (8.0)
Epoch 143/301 | Loss 0.0046 | Win/lose count 10.0/0 (10.0)
Epoch 144/301 | Loss 0.0033 | Win/lose count 17.0/2.0 (15.0)
Epoch 145/301 | Loss 0.0013 | Win/lose count 20.0/1.0 (19.0)
Epoch 146/301 | Loss 0.0027 | Win/lose count 13.0/2.0 (11.0)
Epoch 147/301 | Loss 0.0019 | Win/lose count 21.5/2.0 (19.5)
Epoch 148/301 | Loss 0.0024 | Win/lose count 13.5/1.0 (12.5)
Epoch 149/301 | Loss 0.0033 | Win/lose count 11.0/4.0 (7.0)
Epoch 150/301 | Loss 0.0462 | Win/lose count 15.0/2.0 (13.0)
Epoch 151/301 | Loss 0.0023 | Win/lose count 18.0/4.0 (14.0)
Epoch 152/301 | Loss 0.0031 | Win/lose count 5.5/3.0 (2.5)
Epoch 153/301 | Loss 0.0017 | Win/l

Epoch 273/301 | Loss 0.0023 | Win/lose count 12.5/3.0 (9.5)
Epoch 274/301 | Loss 0.0495 | Win/lose count 19.0/5.0 (14.0)
Epoch 275/301 | Loss 0.0021 | Win/lose count 20.5/2.0 (18.5)
Epoch 276/301 | Loss 0.0026 | Win/lose count 16.0/3.0 (13.0)
Epoch 277/301 | Loss 0.0036 | Win/lose count 11.5/2.0 (9.5)
Epoch 278/301 | Loss 0.0013 | Win/lose count 19.5/3.0 (16.5)
Epoch 279/301 | Loss 0.0033 | Win/lose count 13.0/3.0 (10.0)
Epoch 280/301 | Loss 0.0472 | Win/lose count 11.5/0 (11.5)
Epoch 281/301 | Loss 0.0031 | Win/lose count 17.0/4.0 (13.0)
Epoch 282/301 | Loss 0.0531 | Win/lose count 15.0/4.0 (11.0)
Epoch 283/301 | Loss 0.0520 | Win/lose count 11.0/2.0 (9.0)
Epoch 284/301 | Loss 0.0023 | Win/lose count 20.5/2.0 (18.5)
Epoch 285/301 | Loss 0.0026 | Win/lose count 9.5/4.0 (5.5)
Epoch 286/301 | Loss 0.0028 | Win/lose count 19.5/1.0 (18.5)
Epoch 287/301 | Loss 0.0017 | Win/lose count 21.0/4.0 (17.0)
Epoch 288/301 | Loss 0.0026 | Win/lose count 17.5/4.0 (13.5)
Epoch 289/301 | Loss 0.0032 | W

In [17]:
HTML(display_videos('cnn_train300.mp4'))

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [18]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 10.0/3.0. Average score (7.0)
Win/lose count 10.0/4.0. Average score (6.5)
Win/lose count 23.0/1.0. Average score (11.666666666666666)
Win/lose count 15.0/3.0. Average score (11.75)
Win/lose count 21.0/2.0. Average score (13.2)
Win/lose count 8.5/2.0. Average score (12.083333333333334)
Win/lose count 10.0/4.0. Average score (11.214285714285714)
Win/lose count 14.5/3.0. Average score (11.25)
Win/lose count 3.5/1.0. Average score (10.277777777777779)
Win/lose count 13.0/3.0. Average score (10.25)
Win/lose count 8.0/3.0. Average score (9.772727272727273)
Win/lose count 15.5/2.0. Average score (10.083333333333334)
Win/lose count 18.5/3.0. Average score (10.5)
Win/lose count 14.0/3.0. Average score (10.535714285714286)
Win/lose count 20.5/3.0. Average score (11.0)
Win/lose count 9.0/1.0. Average score (10.8125)
Win/lose count 10.5/3.0. Average score (10.617647058823529)
Win/lose count 8.0/2.0. Average score (10.36111111111111)
Win/lose count 20.5/3.0. Average 

In [19]:
HTML(display_videos('cnn_test30.mp4'))

In [20]:
HTML(display_videos('fc_test30.mp4'))

> The mouse might be stuck in a zone once it emptied of all its cheese. Then it will oscillate in the empty area, avoiding poison but not exploring enough the environment to find other areas with cheese.
>
> With lower temperature the problem is even worst, as the cheese is more sparse, the algorithm won't visit entire areas.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [21]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
                

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
            linear = True
            exponential = False
            if linear:
                agent.set_epsilon(agent.epsilon * (1 - 5 / epoch))
            elif exponential:
                agent.set_epsilon(agent.epsilon ** 0.90)
        
        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

In [22]:
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1, explore=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        self.explore = explore
        
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.malus_position[self.x, self.y] += self.explore
            
        self.t = self.t + 1
        reward = self.board[self.x, self.y] - self.malus_position[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [23]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3, explore=0.01)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore300.mp4'))

Epoch 000/301 | Loss 0.0107 | Win/lose count 3.920000000000001/23.280000000000005 (-19.360000000000003)
Epoch 001/301 | Loss 0.0034 | Win/lose count 11.760000000000003/16.95999999999999 (-5.199999999999987)
Epoch 002/301 | Loss 0.0092 | Win/lose count 5.3900000000000015/24.269999999999985 (-18.879999999999985)
Epoch 003/301 | Loss 0.0067 | Win/lose count 5.3900000000000015/27.52 (-22.13)
Epoch 004/301 | Loss 0.0100 | Win/lose count 2.9400000000000004/27.850000000000005 (-24.910000000000004)
Epoch 005/301 | Loss 0.0073 | Win/lose count 7.350000000000002/22.929999999999993 (-15.579999999999991)
Epoch 006/301 | Loss 0.0053 | Win/lose count 10.780000000000003/18.249999999999996 (-7.4699999999999935)
Epoch 007/301 | Loss 0.0052 | Win/lose count 4.410000000000001/14.950000000000003 (-10.540000000000003)
Epoch 008/301 | Loss 0.0097 | Win/lose count 9.800000000000002/24.910000000000014 (-15.110000000000012)
Epoch 009/301 | Loss 0.0119 | Win/lose count 8.820000000000002/16.079999999999988 (-7.2

Epoch 081/301 | Loss 0.0062 | Win/lose count 13.230000000000004/10.669999999999986 (2.5600000000000183)
Epoch 082/301 | Loss 0.0054 | Win/lose count 16.660000000000004/7.01999999999999 (9.640000000000015)
Epoch 083/301 | Loss 0.0076 | Win/lose count 16.660000000000004/4.82999999999999 (11.830000000000013)
Epoch 084/301 | Loss 0.0052 | Win/lose count 22.049999999999986/5.929999999999993 (16.119999999999994)
Epoch 085/301 | Loss 0.0051 | Win/lose count 21.06999999999999/8.299999999999981 (12.770000000000008)
Epoch 086/301 | Loss 0.0058 | Win/lose count 18.13/6.569999999999987 (11.560000000000013)
Epoch 087/301 | Loss 0.0037 | Win/lose count 21.06999999999999/6.159999999999987 (14.910000000000004)
Epoch 088/301 | Loss 0.0075 | Win/lose count 20.57999999999999/4.959999999999994 (15.619999999999997)
Epoch 089/301 | Loss 0.0088 | Win/lose count 23.029999999999983/4.569999999999989 (18.459999999999994)
Epoch 090/301 | Loss 0.0064 | Win/lose count 18.13/6.6499999999999915 (11.480000000000008)


Epoch 163/301 | Loss 0.0150 | Win/lose count 19.599999999999994/5.459999999999987 (14.140000000000008)
Epoch 164/301 | Loss 0.0512 | Win/lose count 23.519999999999982/3.1499999999999977 (20.369999999999983)
Epoch 165/301 | Loss 0.0049 | Win/lose count 20.57999999999999/7.0099999999999785 (13.570000000000013)
Epoch 166/301 | Loss 0.0040 | Win/lose count 23.519999999999982/6.8799999999999955 (16.639999999999986)
Epoch 167/301 | Loss 0.0045 | Win/lose count 16.170000000000005/4.969999999999993 (11.200000000000014)
Epoch 168/301 | Loss 0.0041 | Win/lose count 20.57999999999999/7.70999999999998 (12.870000000000012)
Epoch 169/301 | Loss 0.0064 | Win/lose count 18.619999999999997/5.619999999999988 (13.00000000000001)
Epoch 170/301 | Loss 0.0049 | Win/lose count 25.479999999999976/2.7799999999999994 (22.699999999999974)
Epoch 171/301 | Loss 0.0150 | Win/lose count 21.06999999999999/3.1699999999999977 (17.89999999999999)
Epoch 172/301 | Loss 0.0212 | Win/lose count 21.06999999999999/7.249999999

Epoch 245/301 | Loss 0.0246 | Win/lose count 20.57999999999999/3.9399999999999973 (16.639999999999993)
Epoch 246/301 | Loss 0.0337 | Win/lose count 24.00999999999998/5.839999999999979 (18.17)
Epoch 247/301 | Loss 0.0048 | Win/lose count 23.029999999999983/9.699999999999964 (13.33000000000002)
Epoch 248/301 | Loss 0.0129 | Win/lose count 10.780000000000003/8.779999999999996 (2.000000000000007)
Epoch 249/301 | Loss 0.0064 | Win/lose count 27.929999999999968/6.89999999999999 (21.02999999999998)
Epoch 250/301 | Loss 0.0054 | Win/lose count 14.210000000000004/6.249999999999984 (7.96000000000002)
Epoch 251/301 | Loss 0.0121 | Win/lose count 19.599999999999994/6.819999999999992 (12.780000000000001)
Epoch 252/301 | Loss 0.0056 | Win/lose count 23.519999999999982/8.279999999999982 (15.24)
Epoch 253/301 | Loss 0.0047 | Win/lose count 14.700000000000005/5.929999999999981 (8.770000000000024)
Epoch 254/301 | Loss 0.0083 | Win/lose count 18.13/4.949999999999983 (13.180000000000016)
Epoch 255/301 | L

In [24]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore30.mp4'))

Win/lose count 9.310000000000002/7.509999999999987. Average score (1.800000000000015)
Win/lose count 23.519999999999982/5.5899999999999865. Average score (9.865000000000006)
Win/lose count 23.519999999999982/6.9399999999999835. Average score (12.103333333333337)
Win/lose count 21.06999999999999/4.83999999999999. Average score (13.135000000000002)
Win/lose count 24.49999999999998/3.719999999999994. Average score (14.663999999999998)
Win/lose count 24.00999999999998/3.8899999999999975. Average score (15.573333333333329)
Win/lose count 26.94999999999997/4.869999999999991. Average score (16.502857142857135)
Win/lose count 25.479999999999976/3.789999999999995. Average score (17.15124999999999)
Win/lose count 25.969999999999974/3.0799999999999996. Average score (17.788888888888877)
Win/lose count 19.109999999999996/8.939999999999984. Average score (17.026999999999994)
Win/lose count 24.989999999999977/4.869999999999994. Average score (17.308181818181808)
Win/lose count 25.479999999999976/5.3

> First, the decreasing $\epsilon$-greedy policy has low impact alone on the quality of the result, as the mouse will more likely take a poison.
> On the overhand, if the explore penality is well set compared to the temperature, the quality of the result increases a lot.
> We can obtain really good performance by combining the two methods.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***